In [2]:
import numpy as np
import re
import string
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
import faiss
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer

c:\Users\rabia\miniconda3\envs\final_rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Encoder İndirme

In [11]:
# Belgeleri vektörlere dönüştürecek bir encoder, rag sisteminin benzerliği ölçmesi için gerekli
# When did World War II end? -> [0.54, -0.21, ...]  (768,)lik vektörlere dönüştürür
encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


## Veri Setini Yükleme ve Düzenleme

In [ ]:
# Küçültülmüş veri setinin hepsini yükleme
dataset_all = load_dataset("nq_open")
print(dataset_all["train"].shape)
print(dataset_all["validation"].shape)

README.md:   0%|          | 0.00/8.77k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/214k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87925 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3610 [00:00<?, ? examples/s]

(87925, 2)
(3610, 2)


In [ ]:
print(dataset_all)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 87925
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 3610
    })
})


In [5]:
# Veri setini yükleme
dataset = load_dataset("nq_open", split="train[:5000]")  # NaturalQuestions veri setinin küçültülmüş versiyonu (NaturalQuestions=150GB)
print(f"Veri kümesi boyutu: {len(dataset)}")


c:\Users\rabia\miniconda3\envs\final_rag_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rabia\.cache\huggingface\hub\datasets--nq_open. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating validation split: 100%|██████████| 3610/3610 [00:00<00:00, 905101.17 examples/s]

Veri kümesi boyutu: 5000


In [6]:
# Veri setini bölme
shuffled_dataset = dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(4000))
eval_dataset = shuffled_dataset.select(range(4000, 5000))

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': eval_dataset
})

print(f"Eğitim veri kümesi boyutu: {len(dataset_dict['train'])}")
print(f"Değerlendirme veri kümesi boyutu: {len(dataset_dict['validation'])}")

Eğitim veri kümesi boyutu: 4000
Değerlendirme veri kümesi boyutu: 1000


In [7]:
print(train_dataset['question'][:2])
print(train_dataset['answer'][:2])

['who is the publisher of the daily independent', 'when did seat belts first appear in cars']
[['Independent Newspapers Limited'], ['1949']]


In [8]:
# Soru cevapları birbiriyle eşleştirerek daha verimli veri kullanımını sağlama

documents = []
for i, example in enumerate(dataset):
    if 'question' in example and 'answer' in example and example['answer']:
        question = example['question']

        # cevap için liste içerisinde birden fazla örnek olduğunda, ['Noah Jupe', 'John Krasinski', 'Emily Blunt', 'Millicent Simmonds'] gibi
        answer = " ".join(example['answer'])

        # Soru ve cevap birlikte olacak şekilde eşleştirilir
        documents.append({
            'id': str(i),
            'question': question,
            'content': f"Question: {question} Answer: {answer}" # embedding yapılıp vektör veri tabanına atılacak asıl metin
        })
print(f"Toplam belge sayısı: {len(documents)}")


Toplam belge sayısı: 5000


In [ ]:
documents[0]['content']

'Question: where did they film hot tub time machine Answer: Fernie Alpine Resort'

In [9]:
# Belgelerden soru-cevapları çekme
document_texts = [doc['content'] for doc in documents]

In [12]:
# Belge embedding'lerini oluşturma
print("Embedding işlemi başladı")
batch_size = 32
document_embeddings = []

for i in range(0, len(document_texts), batch_size):
    batch = document_texts[i:i+batch_size]
    batch_embeddings = encoder.encode(batch, convert_to_tensor=False) # convert_to_tensor=False -> numpy array olarak döndürmesi için (faiss için)
    document_embeddings.append(batch_embeddings)

print(len(document_embeddings))
print(len(document_embeddings[0]))
document_embeddings = np.vstack(document_embeddings)  # faiss'e tek seferde verebilmek için
print(len(document_embeddings))
print(document_embeddings.shape)
print("Embedding işlemi tamamlandı")

Embedding işlemi başladı
157
32
5000
(5000, 768)
Embedding işlemi tamamlandı


## Faiss ile Vektörleri Tutma

In [13]:
# FAISS ile vektör arama için indeks oluşturma
dimension = document_embeddings.shape[1]  # embedding vektörlerinin boyutu : 768
index = faiss.IndexFlatIP(dimension)  # Kullanılan vektör benzerliğinin türünü belirler. IP:Inner product vektörlerin arasındaki açıya göre benzerlik hesaplar(semantik benzerlik için yaygın)

# FAISS iiçin gerekli veri tipi float32
document_embeddings = document_embeddings.astype('float32')

# Normalizasyon yap (kosinüs benzerliği için gerekli)
# [3, 4] -> [0.6, 0.8] haline getirir, yani vektörler birim çember üzerine dizilir, açısal benzerliğin doğru çalışması için
faiss.normalize_L2(document_embeddings)

# İndekse ekle
index.add(document_embeddings)  # vektörler faisse saklanır, arama yaparken yeni bir vektör geldiğinde uzayda en yakın olan bulunur.
print(f"FAISS indeks boyutu: {index.ntotal}")

FAISS indeks boyutu: 5000


## T5 Modelini Yükleme

In [ ]:
# LLM modeli yüklenir
model_name = "google/flan-t5-large"  # Base(250M) yerine large model(3B)
tokenizer = AutoTokenizer.from_pretrained(model_name) # Gelen soru cümlelerini modelin anlayabileceği sayısal formata çevirerek modele input hazırlar
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

c:\Users\rabia\miniconda3\envs\final_rag_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rabia\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regula

## Değerlendirme Metrikleri

In [ ]:
# Değerlendirme metrikleri
def normalize_text(text):
    text = text.lower() # küçük harfe çevir
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text) # noktalama işaretlerini kaldırır
    text = re.sub(r'\b(a|an|the)\b', ' ', text) # belirtilen kelimeleri kaldırır
    text = re.sub(r'\s+', ' ', text).strip()  # fazla boşlukları tek boşluk olarak değiştirir
    return text

def compute_em(prediction, ground_truth): # EM:Exact Match (prediction:'paris', ground truth:'Paris', EM:1)
    return int(normalize_text(prediction) == normalize_text(ground_truth))

def compute_f1(prediction, ground_truth):
    # precision: modelin cevabındaki kelimelerin kaçı doğru?
    # recall: model doğru cevaptaki kelimelerin kaçını bildi?
    pred_tokens = normalize_text(prediction).split()
    gt_tokens = normalize_text(ground_truth).split()

    common = set(pred_tokens) & set(gt_tokens)
    num_same = len(common)

    if num_same == 0 or len(pred_tokens) == 0 or len(gt_tokens) == 0:
        return 0.0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def compute_rouge_l(prediction, ground_truth):
    # Tahminle doğru cevap arasındaki en uzun ortak kelime dizisinde göre puan verir, f1 gibi kelime sayısını değil sırasını da işleme alır
    def lcs(X, Y):
        m = len(X)
        n = len(Y)
        L = [[0] * (n+1) for _ in range(m+1)]

        for i in range(m):
            for j in range(n):
                if X[i] == Y[j]:
                    L[i+1][j+1] = L[i][j] + 1
                else:
                    L[i+1][j+1] = max(L[i+1][j], L[i][j+1])
        return L[m][n]

    pred_tokens = normalize_text(prediction).split()
    gt_tokens = normalize_text(ground_truth).split()

    if not pred_tokens or not gt_tokens:
        return 0.0

    lcs_len = lcs(pred_tokens, gt_tokens)
    recall = lcs_len / len(gt_tokens) if gt_tokens else 0
    precision = lcs_len / len(pred_tokens) if pred_tokens else 0

    if recall + precision == 0:
        return 0.0

    return (2 * recall * precision) / (recall + precision)

def get_best_score(metric_func, prediction, ground_truths):
    # tahmin için en yüksek puanı alan ground truth'u kullanır
    if isinstance(ground_truths, list):
        return max(metric_func(prediction, gt) for gt in ground_truths)
    else:
        return metric_func(prediction, ground_truths)


## Cevap Üretme

In [ ]:
def generate_answer(question, k=5, return_docs=False):  # k'yı 3'ten 5'e çıkardık
    # Soru vektör haline getirilir
    query_embedding = encoder.encode([question], convert_to_tensor=False).astype('float32')
    faiss.normalize_L2(query_embedding)

    # En yakın k belgeyi bul
    distances, indices = index.search(query_embedding, k=k)

    # Belge içeriklerini al ve mesafelere göre sırala
    retrieved_docs = [(distances[0][i], document_texts[indices[0][i]], indices[0][i]) for i in range(len(indices[0]))]
    retrieved_docs.sort(key=lambda x: x[0], reverse=True)  # Benzerliğe göre sırala

    # Mesafeleri ağırlık olarak kullanması sağlanır
    context = "\n\n".join([f"Document {i+1} (Relevance: {dist:.2f}, Index: {idx}): {text}"
                           for i, (dist, text, idx) in enumerate(retrieved_docs)])

    # input promptu
    input_text = f"""Below are some documents that might contain information to answer the question.
                  {context}
                  Based on the above documents, answer the following question accurately and concisely.
                  Question: {question}
                  Answer:"""

    # Tokenize ve generate
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True) # return_tensors="pt"->generate() tensor aldığı için, flan-t5-large için max_length=1024, eğer bu değer aşılırsa truncation=True ile kırpma uygulanır.

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=64,  # Modelin max kaç token ile cevap üreteceği, cevap uzunluğu (64 token yaklaşık olarak 40-50 kelimeye denk gelir)
            num_beams=4,        # Beam search: Her adımda 4 olası token ile devam eder, sonunda en yüksek olasılıklı olanla cümleyi oluşturur. Daha iyi kalite ama daha yavaş sonuç üretir
            do_sample=False     # Modelin her çağrısında farklı cevap vermesini engeller, kararlılık sağlar
        )

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip() # cevaplar token halinde oluşturulduğu için bunları anlaşılabilir hale dönüştürür. skip_special_tokens=True ile <pad>, </s> gibi modelin iç tokenlarını temizler

    if return_docs:
        return prediction, retrieved_docs
    else:
        return prediction

In [ ]:
test_question = "What is the capital of Turkey?"
test_answer, test_docs = generate_answer(test_question, return_docs=True)

print(f"\nTest Sorusu: {test_question}")
print(f"Model Cevabı: {test_answer}")
print("\nGetirilen Belgeler:")
for i, (similarity, text, idx) in enumerate(test_docs):
    print(f"\nBelge {i+1} (Benzerlik: {similarity:.4f}, İndeks: {idx}):")
    print(f"İçerik: {text}")


Test Sorusu: What is the capital of Turkey?
Model Cevabı: Istanbul

Getirilen Belgeler:

Belge 1 (Benzerlik: 0.4957, İndeks: 3616):
İçerik: Question: what country's flag has a moon and star Answer: Turkey

Belge 2 (Benzerlik: 0.4908, İndeks: 3144):
İçerik: Question: who claimed a homeland in parts of turkey iran and iraq Answer: Kurdish people

Belge 3 (Benzerlik: 0.4390, İndeks: 1188):
İçerik: Question: ancient area of asia minor including aegean islands Answer: Anatolia

Belge 4 (Benzerlik: 0.4260, İndeks: 4455):
İçerik: Question: where was the city of troy located at Answer: Anatolia in modern Turkey

Belge 5 (Benzerlik: 0.4257, İndeks: 4944):
İçerik: Question: how much territory did the ottoman empire cover Answer: 2,000,000 sq mi


In [ ]:
questions = [
    "What is the capital of France?",
    "When was the moon landing?",
    "Who invented the telephone?"
]

for q in questions:
    print("Q:", q)
    print("A:", generate_answer(q), "\n")


Q: What is the capital of France?
A: Paris 

Q: When was the moon landing?
A: 20 July 1969 

Q: Who invented the telephone?
A: The telephone was invented by Alexander Graham Bell 



## Değerlendirme

In [ ]:
num_eval_samples = 100  # Değerlendirme için daha fazla örnek kullan
eval_samples = eval_dataset.select(range(min(num_eval_samples, len(eval_dataset))))

em_scores = []
f1_scores = []
rouge_scores = []

for i, example in enumerate(tqdm(eval_samples)):
    question = example['question']
    ground_truth = example['answer']

    prediction = generate_answer(question, k=5)

    # En iyi skor seçilir
    em_scores.append(get_best_score(compute_em, prediction, ground_truth))
    f1_scores.append(get_best_score(compute_f1, prediction, ground_truth))
    rouge_scores.append(get_best_score(compute_rouge_l, prediction, ground_truth))

    if i < 5:  # İlk 5 örnek için ayrıntılı çıktı göster
        print(f"\nÖrnek {i+1}:")
        print(f"Soru: {question}")
        print(f"Gerçek Cevap: {ground_truth}")
        print(f"Tahmin: {prediction}")
        print(f"EM: {em_scores[-1]}, F1: {f1_scores[-1]:.2f}, ROUGE-L: {rouge_scores[-1]:.2f}")

# Ortalama skorları yazdır
print("\n=== SONUÇLAR ===")
print(f"Ortalama EM: {sum(em_scores)/len(em_scores):.4f}")
print(f"Ortalama F1: {sum(f1_scores)/len(f1_scores):.4f}")
print(f"Ortalama ROUGE-L: {sum(rouge_scores)/len(rouge_scores):.4f}")


  0%|          | 0/100 [00:00<?, ?it/s]


Örnek 1:
Soru: who builds the walls of thebes with song
Gerçek Cevap: ['Amphion']
Tahmin: Amphion
EM: 1, F1: 1.00, ROUGE-L: 1.00

Örnek 2:
Soru: who played the brain in pinky and the brain
Gerçek Cevap: ['Rob Paulsen']
Tahmin: Rob Paulsen
EM: 1, F1: 1.00, ROUGE-L: 1.00

Örnek 3:
Soru: who played jerry lee lewis in great balls of fire
Gerçek Cevap: ['Dennis Quaid']
Tahmin: Dennis Quaid
EM: 1, F1: 1.00, ROUGE-L: 1.00

Örnek 4:
Soru: where is the u21 euro championships being held
Gerçek Cevap: ['Poland']
Tahmin: Poland
EM: 1, F1: 1.00, ROUGE-L: 1.00

Örnek 5:
Soru: when did isaiah thomas go to the cavs
Gerçek Cevap: ['August 22, 2017']
Tahmin: August 22, 2017
EM: 1, F1: 1.00, ROUGE-L: 1.00

=== SONUÇLAR ===
Ortalama EM: 0.8800
Ortalama F1: 0.9406
Ortalama ROUGE-L: 0.9406


# İnteraktif RAG Sistemi

In [ ]:
# İnteraktif RAG sistemi oluşturma
def interactive_rag():
    print("\n" + "="*50)
    print("İNTERAKTİF SORU-CEVAP SİSTEMİ")
    print("="*50)
    print("Sorularınızı yazın, çıkmak için 'q' veya 'exit' yazın.")
    print("Belgeleri görmek için soru sonuna '!' ekleyin (örn: 'What is DNA?!')")
    print("="*50)

    while True:
        question = input("\nSorunuz: ")

        # Çıkış kontrolü
        if question.lower() in ['q', 'exit', 'quit', 'çık', 'çıkış']:
            print("Sistemden çıkılıyor...")
            break

        # Belgeleri göster/gösterme seçeneği
        show_docs = False
        if question.endswith('!'):
            show_docs = True
            question = question[:-1]  # '!' işaretini kaldır

        start_time = time.time()

        try:
            if show_docs:
                answer, docs = generate_answer(question, return_docs=True)
                print(f"\nCevap: {answer}")

                print("\nKullanılan Belgeler:")
                for i, (similarity, text, idx) in enumerate(docs):
                    print(f"\nBelge {i+1} (Benzerlik: {similarity:.4f}, İndeks: {idx}):")
                    print(f"İçerik: {text}")
            else:
                answer = generate_answer(question)
                print(f"\nCevap: {answer}")

            elapsed_time = time.time() - start_time
            print(f"\nCevap süresi: {elapsed_time:.2f} saniye")

            # Veri setinde benzer soru kontrolü
            if question.endswith('?'):
                words = question.lower().replace('?', '').split()
                if len(words) >= 2:  # En az iki kelimeli sorular için kontrol
                    # Sorudaki anahtar kelimeleri alıp veri setinde benzer soru ara
                    keywords = [w for w in words if len(w) > 3 and w not in
                               ['what', 'where', 'when', 'which', 'who', 'how', 'does', 'is', 'are', 'was', 'were']]

                    if keywords:
                        print(f"\nVeri setinde '{', '.join(keywords[:2])}' ile ilgili soru aranıyor...")
                        matching_questions = []

                        # En önemli iki anahtar kelimeyi kullan
                        key1, key2 = keywords[:2] if len(keywords) >= 2 else (keywords[0], '')

                        for ex in dataset:
                            if key1 in ex['question'].lower() and (not key2 or key2 in ex['question'].lower()):
                                matching_questions.append(ex)
                                if len(matching_questions) >= 3:  # En fazla 3 eşleşme bul
                                    break

                        if matching_questions:
                            print(f"Veri setinde {len(matching_questions)} benzer soru bulundu:")
                            for i, match in enumerate(matching_questions):
                                print(f"\nBenzer Soru {i+1}: {match['question']}")
                                print(f"Cevap: {match['answer']}")
                        else:
                            print("Veri setinde benzer soru bulunamadı.")

        except Exception as e:
            print(f"Hata oluştu: {str(e)}")

In [ ]:
test_question = "What is the capital of Turkey?"
test_answer, test_docs = generate_answer(test_question, return_docs=True)

print(f"\nTest Sorusu: {test_question}")
print(f"Model Cevabı: {test_answer}")
print("\nGetirilen Belgeler:")
for i, (similarity, text, idx) in enumerate(test_docs):
    print(f"\nBelge {i+1} (Benzerlik: {similarity:.4f}, İndeks: {idx}):")
    print(f"İçerik: {text}")


Test Sorusu: What is the capital of Turkey?
Model Cevabı: Istanbul

Getirilen Belgeler:

Belge 1 (Benzerlik: 0.4957, İndeks: 3616):
İçerik: Question: what country's flag has a moon and star Answer: Turkey

Belge 2 (Benzerlik: 0.4908, İndeks: 3144):
İçerik: Question: who claimed a homeland in parts of turkey iran and iraq Answer: Kurdish people

Belge 3 (Benzerlik: 0.4390, İndeks: 1188):
İçerik: Question: ancient area of asia minor including aegean islands Answer: Anatolia

Belge 4 (Benzerlik: 0.4260, İndeks: 4455):
İçerik: Question: where was the city of troy located at Answer: Anatolia in modern Turkey

Belge 5 (Benzerlik: 0.4257, İndeks: 4944):
İçerik: Question: how much territory did the ottoman empire cover Answer: 2,000,000 sq mi


In [ ]:
dataset['question'][0]

'where did they film hot tub time machine'

In [ ]:
import time
print("\nİnteraktif sistem başlatılıyor...")
interactive_rag()


İnteraktif sistem başlatılıyor...

İNTERAKTİF SORU-CEVAP SİSTEMİ
Sorularınızı yazın, çıkmak için 'q' veya 'exit' yazın.
Belgeleri görmek için soru sonuna '!' ekleyin (örn: 'What is DNA?!')

Sorunuz: when did they film hot tub time machine!

Cevap: Fernie Alpine Resort

Kullanılan Belgeler:

Belge 1 (Benzerlik: 0.6929, İndeks: 0):
İçerik: Question: where did they film hot tub time machine Answer: Fernie Alpine Resort

Belge 2 (Benzerlik: 0.4383, İndeks: 883):
İçerik: Question: when was fast times at ridgemont high made Answer: 1982

Belge 3 (Benzerlik: 0.3974, İndeks: 3199):
İçerik: Question: what year was the future in back to the future Answer: 2015

Belge 4 (Benzerlik: 0.3795, İndeks: 2003):
İçerik: Question: when did the cubs win in back to the future Answer: 2015

Belge 5 (Benzerlik: 0.3531, İndeks: 3495):
İçerik: Question: time after time song was in what movie Answer: John Tucker Must Die

Cevap süresi: 14.51 saniye

Sorunuz: q
Sistemden çıkılıyor...
